In [30]:
import lightgbm as lgb
from sklearn import metrics
import category_encoders as ce

from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import xgboost as xgb

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

In [31]:
df = pd.read_csv('../data/train.csv',dtype={'tipodepropiedad':'category','ciudad':'category','provincia':'category'})
df['fecha'] = pd.to_datetime(df['fecha'])
df['anio'] = df['fecha'].dt.year
df["mes"] = df['fecha'].dt.month
df["dia"] = df['fecha'].dt.day

df = df.drop(columns=["lat","lng"])

In [32]:
def imputar(df,cols):
    imp = SimpleImputer()
    df_result = df.copy()
    for x in cols:
        df_result[x]=imp.fit_transform(df[[x]])
    return df_result

In [33]:
def normalizar(df):
    min_max_scaler = MinMaxScaler()
    cols = df._get_numeric_data().columns
    sub_df = df[cols]
    res = df.drop(columns=cols)
    x_scaled = min_max_scaler.fit_transform(sub_df)
    x_scaled = pd.DataFrame(x_scaled,columns=cols)
    return res.join(x_scaled)

In [34]:
imp = SimpleImputer(strategy="constant")
df["provincia"] = imp.fit_transform(df[["provincia"]])
df["ciudad"] = imp.fit_transform(df[["ciudad"]])

In [35]:
df['ciudad-provincia'] = df[['ciudad', 'provincia']].apply(lambda x: ', '.join(x), axis=1)

In [36]:
#Columnas numericas
data_cols = df._get_numeric_data().columns

#Completo los nulos
df = imputar(df,data_cols)
#Normalizo

#Hay veces que no me conviene

#df = normalizar(df)

In [37]:
oneHot = ['tipodepropiedad','provincia']
label = ['ciudad','ciudad-provincia']

encoder = ce.OneHotEncoder()

encoded = encoder.fit_transform(df[oneHot])
data = df.join(encoded)

encoder = ce.BinaryEncoder()

encoded = encoder.fit_transform(df[label])
data = data.join(encoded)

In [39]:
df = data

In [40]:
import re
def contiene_frase(texto, frase):
    if re.search(frase, texto,re.IGNORECASE):
        return True
    return False

contiene_frase=np.vectorize(contiene_frase)

In [41]:
def cantidad_de_public(data,colum,palabra):
    return data[contiene_frase(data[colum],palabra)].dia.count()

def serie_que_contiene_palabra(data,colum,palabra):
        return contiene_frase(data[colum],palabra)

# Completo los datos faltantes

# Busco en las descripciones

In [42]:
busqueda = "descripcion"
df1 = df.copy()
df1[busqueda] = df1[busqueda].fillna(value="-")
df1.shape

(240000, 104)

In [43]:
palabra = "centro"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

25889

In [44]:
palabra = "patio"
palabra2 = "jardin"
palabra3 = "jardín"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)|serie_que_contiene_palabra(df1,busqueda,palabra2)|serie_que_contiene_palabra(df1,busqueda,palabra3)
df1[palabra].value_counts()[1]

100824

In [45]:
palabra = "hermosa"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

21198

In [46]:
palabra = "grande"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

11163

In [47]:
palabra = "universidad"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

4221

In [48]:
palabra = "ilum"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)


20875

In [49]:
palabra = "seguridad"
palabra2 = "vigilancia"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)|serie_que_contiene_palabra(df1,busqueda,palabra2)
df1[palabra].value_counts()[1]

52199

In [50]:
palabra = "tienda"
palabra2 = "comercial"
palabra3 = "local"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)|serie_que_contiene_palabra(df1,busqueda,palabra2)|serie_que_contiene_palabra(df1,busqueda,palabra3)
df1[palabra].value_counts()[1]

39496

In [51]:
palabra = "comedor"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

127434

In [52]:
palabra = "lujo"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

13816

In [53]:
palabra = "servicio"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

87104

In [54]:
palabra = "remodelada"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

6379

In [55]:
palabra = "autos"
palabra2 = "auto"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)|serie_que_contiene_palabra(df1,busqueda,palabra2)
df1[palabra].value_counts()[1]

78290

# Busco en las direcciones

In [56]:
busqueda = "direccion"

df1[busqueda] = df1[busqueda].fillna(value="-")

In [57]:
palabra = "avenida"
palabra2 = "av"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)|serie_que_contiene_palabra(df1,busqueda,palabra2)
df1[palabra].value_counts()[1]

20304

# Creo el CSV

In [58]:
df1.shape

(240000, 118)

In [59]:
df1.to_csv("../data/dataset_imputado.csv", index=False)

In [60]:
df1

,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,fecha,ciudad-provincia,id,antiguedad,...,universidad,ilum,seguridad,tienda,comedor,lujo,servicio,remodelada,autos,avenida
0,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,2015-08-23,"Benito Juárez, Distrito Federal",0.846999,0.101451,...,False,False,False,False,True,False,False,False,False,True
1,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,2013-06-28,"La Magdalena Contreras, Distrito Federal",0.178201,0.125000,...,False,False,False,False,True,False,True,False,True,True
2,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,2015-10-17,"Tonalá, Jalisco",0.826616,0.062500,...,False,False,True,True,False,False,True,False,True,False
3,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,2012-03-09,"Zinacantepec, Edo. de México",0.696891,0.012500,...,False,False,True,True,False,False,False,False,False,False
4,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,2016-06-07,"Zapopan, Jalisco",0.619991,0.125000,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239995,bonita casas de 2 recamaras a 10 minutos del c...,vendo casa en bosques de ica residencial a 10 ...,Casa,BOSQUES,Zinacantepec,Edo. de México,2015-02-08,"Zinacantepec, Edo. de México",0.399596,0.000000,...,False,False,True,False,True,False,False,False,True,False
239996,casa en condominio a 10 min. del centro de toluca,"casa con un jardin amplio, un cuarto de servic...",Casa,Filiberto Navas 325,Toluca,Edo. de México,2014-07-10,"Toluca, Edo. de México",0.863929,0.000000,...,False,False,False,True,False,False,True,False,True,True
239997,nicolas san juan,"departamento con excelente ubicación, muy cerc...",Apartamento,Nicolas San Juan,Benito Juárez,Distrito Federal,2015-03-03,"Benito Juárez, Distrito Federal",0.439773,0.250000,...,True,False,True,False,True,False,True,False,True,False
239998,casa sola. javier rojo gomez.,"casa sola, dividida en cuatro departamentos de...",Casa,Javier Rojo Gomez 120,Iztapalapa,Distrito Federal,2014-12-26,"Iztapalapa, Distrito Federal",0.489557,0.250000,...,False,False,False,False,False,False,False,False,False,True
